In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
"""from google.colab import files
uploaded = files.upload()"""

Saving doctor.txt to doctor.txt


In [3]:
file = open("patient.txt", "r", encoding = "utf8")

In [4]:
lines = []
for i in file:
    lines.append(i)

In [5]:
lines

["I came to see you because my GP sent me to see you didn't he?He said come and see a trick cyclist cuz then you'll be all right.Yeah, he not he not had time to listen to me.You see?Okay.He's a GP.He's a doctor but I don't think he's very clever.He's not very clever because he's not okay getting to getting to this.You see won't let let me talk about this.He sort of listen to it sometimes but then sometimes he doesn'tWell, I've come to see you in the emergency clinic today.So that's why I'm here.Is that right?Yeah, cuz I got to tell you about this.There's important stuff in there really important stuff.Okay.Well before we start with that I come to see you.Yeah name name is a game.I don't bother with names.That's just too good, right?Okay.Call me Audrey.How old is Odell you're only as old as the woman you feel aren't you?I feel quite young and better than most women in their 20s.My ID would say I am 35 but I feel 23 and free free food food for the soul a nice bit of Soul.Yeah, I write a 

In [6]:
# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

In [7]:
import gensim.utils 
p = gensim.utils.simple_preprocess(data)

In [8]:
p[:10]

['came', 'to', 'see', 'you', 'because', 'my', 'gp', 'sent', 'me', 'to']

In [9]:
len(p)

182751

In [10]:
#Downloading nltk data
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NMR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
#print(stop_words)

In [12]:
#words = nltk.word_tokenize(sentence)
words = [w for w in p if w not in stop_words]

In [13]:
words

['came',
 'see',
 'gp',
 'sent',
 'see',
 'said',
 'come',
 'see',
 'trick',
 'cyclist',
 'cuz',
 'right',
 'yeah',
 'time',
 'listen',
 'see',
 'okay',
 'gp',
 'doctor',
 'think',
 'clever',
 'clever',
 'okay',
 'getting',
 'getting',
 'see',
 'let',
 'let',
 'talk',
 'sort',
 'listen',
 'sometimes',
 'sometimes',
 'twell',
 'come',
 'see',
 'emergency',
 'clinic',
 'today',
 'right',
 'yeah',
 'cuz',
 'got',
 'tell',
 'important',
 'stuff',
 'really',
 'important',
 'stuff',
 'okay',
 'well',
 'start',
 'come',
 'see',
 'yeah',
 'name',
 'name',
 'game',
 'bother',
 'names',
 'good',
 'right',
 'okay',
 'call',
 'audrey',
 'old',
 'odell',
 'old',
 'woman',
 'feel',
 'feel',
 'quite',
 'young',
 'better',
 'women',
 'id',
 'would',
 'say',
 'feel',
 'free',
 'free',
 'food',
 'food',
 'soul',
 'nice',
 'bit',
 'soul',
 'yeah',
 'write',
 'big',
 'fish',
 'fantast',
 'hungry',
 'right',
 'food',
 'food',
 'food',
 'starving',
 'able',
 'get',
 'food',
 'let',
 'okay',
 'let',
 'get',


In [12]:
len(words)

35983

In [13]:
# Convert list to string
data = ""
for i in lines:
  data = ' '. join(words) 

In [14]:
data[:100]

'psychiatrist department yeah yeah come see us trick cyclist psychiatrist right clarify psychiatrist okay gp asked see ask would like call name okay psychiatrist okay audrey ask old audrey right okay old would make audrey okay tell bit rightand long going say long would say weekam right thinking normally different job potter business sister okay audrey beyondwho called said bonkers goodness said get fuck away bonkers right yeah right okay tell tell going number done numbers leave good cure world hungerokay going work okay take back step said god tells tell bit talks okay alright god talks god talks clarify talk like talking boys coming ears feel like voice come ears comes mouth voice comes mouth goes ear right yeah yeah voice seems hear ears voice seems come ear rather inside head big guy like call sir okay kind things talking kind things say showing replies yet cuz close god probably know probably met stuff write sounds though keeping really busy sleeping time sleep last time good nig

In [15]:
#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]

'psychiatrist department yeah yeah come see us trick cyclist psychiatrist right clarify psychiatrist okay gp asked see ask would like call name okay psychiatrist okay audrey ask old audrey right okay old would make audrey okay tell bit rightand long going say long would say weekam right thinking normally different job potter business sister okay audrey beyondwho called said bonkers goodness said get fuck away bonkers right yeah right okay tell tell going number done numbers leave good cure world '

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[481, 889, 6, 6, 59, 22, 153, 2450, 2451, 481, 14, 890, 481, 18, 2452]

In [17]:
len(sequence_data)

35983

In [18]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

5265


In [19]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  35980


array([[ 481,  889,    6,    6],
       [ 889,    6,    6,   59],
       [   6,    6,   59,   22],
       [   6,   59,   22,  153],
       [  59,   22,  153, 2450],
       [  22,  153, 2450, 2451],
       [ 153, 2450, 2451,  481],
       [2450, 2451,  481,   14],
       [2451,  481,   14,  890],
       [ 481,   14,  890,  481]])

In [20]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [21]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[ 481  889    6]
 [ 889    6    6]
 [   6    6   59]
 [   6   59   22]
 [  59   22  153]
 [  22  153 2450]
 [ 153 2450 2451]
 [2450 2451  481]
 [2451  481   14]
 [ 481   14  890]]
Response:  [   6   59   22  153 2450 2451  481   14  890  481]


In [22]:
y = to_categorical(y, num_classes=vocab_size)
y[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             52650     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 5265)              5270265   
                                                                 
Total params: 18,371,915
Trainable params: 18,371,915
Non-trainable params: 0
_________________________________________________________________


In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy",metrics='accuracy',optimizer=Adam(learning_rate=0.01))
model.fit(X, y, epochs=100, batch_size=64, callbacks=[checkpoint])

Epoch 1/100
563/563 [==============================] - ETA: 0s - loss: 7.0815 - accuracy: 0.0302
Epoch 00001: loss improved from inf to 7.08153, saving model to next_words.h5
563/563 [==============================] - 291s 511ms/step - loss: 7.0815 - accuracy: 0.0302
Epoch 2/100
416/563 [=====================>........] - ETA: 1:15 - loss: 6.9698 - accuracy: 0.0315

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: warm welcome everyone
['warm', 'welcome', 'everyone']
kilometres
Enter your line: 0
Execution completed.....


In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')
